In [12]:
#-- a script to fetch coordinates of each building
#-- Filip Biljecki <filip@nus.edu.sg>
#-- 2019-08

import pandas as pd
import json
import time
import requests

In [18]:
df=pd.read_csv(r'C:\WORK_IAAC\submissions\Internet of Buildings\IoB_WIP\IoB_Singa\hdb-property-information\hdb-property-information.csv',sep=',')
df.head()

,blk_no,street,max_floor_lvl,year_completed,residential,commercial,market_hawker,miscellaneous,multistorey_carpark,precinct_pavilion,...,3room_sold,4room_sold,5room_sold,exec_sold,multigen_sold,studio_apartment_sold,1room_rental,2room_rental,3room_rental,other_room_rental
0,1,BEACH RD,16,1970,Y,Y,N,N,N,N,...,138,1,2,0,0,0,0,0,0,0
1,1,BEDOK STH AVE 1,14,1975,Y,N,N,Y,N,N,...,204,0,2,0,0,0,0,0,0,0
2,1,CANTONMENT RD,2,2010,N,Y,N,N,N,N,...,0,0,0,0,0,0,0,0,0,0
3,1,CHAI CHEE RD,15,1982,Y,N,N,N,N,N,...,0,10,92,0,0,0,0,0,0,0
4,1,CHANGI VILLAGE RD,4,1975,Y,Y,N,N,N,N,...,54,0,1,0,0,0,0,0,0,0


In [3]:
df.shape

(12674, 24)

In [7]:
df_col = df.columns[:24]
all_att = df_col
print(all_att)

Index(['blk_no', 'street', 'max_floor_lvl', 'year_completed', 'residential',
       'commercial', 'market_hawker', 'miscellaneous', 'multistorey_carpark',
       'precinct_pavilion', 'bldg_contract_town', 'total_dwelling_units',
       '1room_sold', '2room_sold', '3room_sold', '4room_sold', '5room_sold',
       'exec_sold', 'multigen_sold', 'studio_apartment_sold', '1room_rental',
       '2room_rental', '3room_rental', 'other_room_rental'],
      dtype='object')


In [13]:
df['blk_no'].nunique()

3285

In [19]:
# Read the HDB data from the CSV
blocks = df

#-- create a new dictionary with block id, address, and coordinate; or load a previous one if generated
try:
    with open('C:\WORK_IAAC\submissions\Internet of Buildings\IoB_WIP\IoB_Singa\hdb-property-information\blocks_coordinates.json', 'r') as f:
        blocks_coordinates = json.load(f)
    print("Loaded", len(blocks_coordinates), "entries from previous file.")
except:
    blocks_coordinates = {}

In [20]:
blocks_coordinates

{}

In [22]:
# Read the HDB data from the CSV
blocks = df

#-- create a new dictionary with block id, address, and coordinate; or load a previous one if generated
try:
    with open('blocks_coordinates.json', 'r') as f:
        blocks_coordinates = json.load(f)
    print("Loaded", len(blocks_coordinates), "entries from previous file.")
except:
    blocks_coordinates = {}

#-- list of HDB attributes we have in the CSV
all_att = df_col

#-- counter
ids = 0

#-- for each block
for i, j in blocks.iterrows():
    ids += 1
    #-- by default there is no location
    location = None
    #-- count how many features that are buildings are returned
    h = 0
    #-- construct the address for the geocoding API
    address = str(j['blk_no']) + ' ' + str(j['street'])
    print(i, '\t', str(j['blk_no']) + ' ' + str(j['street']))
    if i in blocks_coordinates:
        print("\tAlready fetched, skipping")
        continue

    #-- geocoder query
    for attempt in range(10):
        #-- max. 250 requests per second are allowed, so let's pause every 10ms not to exceed 100 requests per second
        time.sleep(0.010)
        try:
            #-- fetch the location of the block
            #-- no authentication isneeded for this functionality of the API
            response = requests.get('https://developers.onemap.sg/commonapi/search?searchVal=%s&returnGeom=Y&getAddrDetails=Y' % (address), auth=('user', 'password'))
            location = response.json()
            #-- thank you OneMap
        except:
            continue
        else:
            break
    else:
        print('10 attempts failed')

    if location:
        print('\tThere are', location['found'], 'result(s)')
        if location['found'] == 0:
            #-- if nothing is found (rarely happens)
            blocks_coordinates[i] = None
            continue
        #-- we are feeling lucky so we will just take the first result into consideration
        l = location['results'][0]
        #-- save the information
        blocks_coordinates[i] = {
            'blk_no' : str(j['blk_no']),
            'street' : str(j['street']),
            'address' : address,
            'latitude' : l['LATITUDE'],
            'longitude' : l['LONGITUDE']
            }
        #-- save the HDB attributes
        for att in all_att:
            blocks_coordinates[i]['hdb_'+att] = str(j[att])

    else:
        print('Address not found')
        blocks_coordinates[i] = None

    #-- this may take some time, so let's save the data occasionally
    if ids % 100 == 0:
        with open('blocks_coordinates.json', 'w') as f:
            json.dump(blocks_coordinates, f)

with open('blocks_coordinates.json', 'w') as f:
        json.dump(blocks_coordinates, f)

0 	 1 BEACH RD
	There are 3 result(s)
1 	 1 BEDOK STH AVE 1
	There are 2 result(s)
2 	 1 CANTONMENT RD
	There are 3 result(s)
3 	 1 CHAI CHEE RD
	There are 1 result(s)
4 	 1 CHANGI VILLAGE RD
	There are 3 result(s)
5 	 1 DELTA AVE
	There are 1 result(s)
6 	 1 DOVER RD
	There are 2 result(s)
7 	 1 EUNOS CRES
	There are 1 result(s)
8 	 1 EVERTON PK
	There are 2 result(s)
9 	 1 GEYLANG SERAI
	There are 6 result(s)
10 	 1 GHIM MOH RD
	There are 1 result(s)
11 	 1 HAIG RD
	There are 2 result(s)
12 	 1 HOLLAND CL
	There are 1 result(s)
13 	 1 HOUGANG AVE 3
	There are 1 result(s)
14 	 1 JLN KUKOH
	There are 2 result(s)
15 	 1 JLN BT MERAH
	There are 3 result(s)
16 	 1 JOO CHIAT RD
	There are 2 result(s)
17 	 1 KG KAYU RD
	There are 1 result(s)
18 	 1 LOR LEW LIAN
	There are 1 result(s)
19 	 1 LOR 7 TOA PAYOH
	There are 1 result(s)
20 	 1 MARINE TER
	There are 1 result(s)
21 	 1 MARSILING DR
	There are 2 result(s)
22 	 1 MAUDE RD
	There are 2 result(s)
23 	 1 PINE CL
	There are 1 result(s)
24 

In [ ]:
import json
from shapely.geometry import Point, shape
import os

#-- open the OSM dump
with open('singapore-latest.geojson', 'r') as osm_file:
    osm_data = json.load(osm_file)['features']
print(len(osm_data))
osm_buildings = []
#-- filter out features that are not buildings
for f in osm_data:
    if 'building' in f['properties']:
        osm_buildings.append(f)
print(len(osm_buildings))

#-- load the geocoding data generated in the previous script
with open('blocks_coordinates.json', 'r') as f:
    blocks_coordinates = json.load(f)

#-- create a new dictionary to store OSM data, or load the partially generated one (saves time if the script crashes)
try:
    with open('osm_entry.json', 'r') as f:
        osm_entry = json.load(f)
    print("Loaded", len(osm_entry), "entries from previous file.")
except:
    osm_entry = {}

#-- counter
i = 0

for block in blocks_coordinates:
    print(block)
    #-- skip those already stored
    if str(i) in osm_entry:
        print("\tAlready fetched, skipping")
        i += 1
        continue
    else:
        i += 1
    #-- skip those blocks for which the address is not known
    if not blocks_coordinates[block]:
        continue
    #-- convert the lat/lon info into shapely Points
    point = Point(float(blocks_coordinates[block]['longitude']), float(blocks_coordinates[block]['latitude']))
    #-- find the nearest feature (usually we would use point in polygon, but some points from OneMap are outside the OSM polygon)
    d = 1000
    nearest = None
    #-- iterate all buildings and calculate distance. Yes, it's not the most efficient way. TODO: make it more efficient
    for b in osm_buildings:
        gj = b['geometry']
        osm_shape = shape(gj)
        try:
            d_ = osm_shape.distance(point)
            if d_ < d:
                d = d_
                nearest = b
                #-- if the point is in the polygon then the distance is zero, so let's just skip all other polygons
                if d == 0:
                    continue
        except:
            pass

    #-- save the most likely OSM footprint
    osm_entry[block] = nearest
    #-- save the file occasionally
    if i % 100 == 0:
        with open('osm_entry.json', 'w') as f:
            json.dump(osm_entry, f)

with open('osm_entry.json', 'w') as f:
    json.dump(osm_entry, f)

111889
111889
Loaded 1100 entries from previous file.
0
	Already fetched, skipping
1
	Already fetched, skipping
2
	Already fetched, skipping
3
	Already fetched, skipping
4
	Already fetched, skipping
5
	Already fetched, skipping
6
	Already fetched, skipping
7
	Already fetched, skipping
8
	Already fetched, skipping
9
	Already fetched, skipping
10
	Already fetched, skipping
11
	Already fetched, skipping
12
	Already fetched, skipping
13
	Already fetched, skipping
14
	Already fetched, skipping
15
	Already fetched, skipping
16
	Already fetched, skipping
17
	Already fetched, skipping
18
	Already fetched, skipping
19
	Already fetched, skipping
20
	Already fetched, skipping
21
	Already fetched, skipping
22
	Already fetched, skipping
23
	Already fetched, skipping
24
	Already fetched, skipping
25
	Already fetched, skipping
26
	Already fetched, skipping
27
	Already fetched, skipping
28
	Already fetched, skipping
29
	Already fetched, skipping
30
	Already fetched, skipping
31
	Already fetched, skipp